## K-Means Clustering Algorithm

**K-Means** is one of the most widely used algorithms in machine learning, particularly in **unsupervised learning**. The goal of K-Means is to partition the dataset into $K$ distinct, non-overlapping clusters, where each cluster is represented by its center (centroid).



### Steps of the K-Means Algorithm:

1. **Initialization**: Randomly select $K$ data points as the initial cluster centroids.
2. **Assignment**: Assign each data point to the nearest centroid using Euclidean distance.
3. **Update**: Compute new centroids as the mean of the data points assigned to each cluster.
4. **Repeat** steps 2 and 3 until the centroids no longer change significantly (or for a fixed number of iterations).



### Objective Function:

The K-Means algorithm aims to minimize the total within-cluster variance, which is calculated as:

$$
J = \sum_{i=1}^{K} \sum_{x \in C_i} \| x - \mu_i \|^2
$$

- $C_i$: the $i$-th cluster  
- $\mu_i$: the centroid of cluster $i$  
- $\| x - \mu_i \|^2$: the squared Euclidean distance between point $x$ and its cluster centroid



### Advantages:
- Simple and fast
- Easy to understand and implement

### Disadvantages:
- Requires specifying the number of clusters $K$ in advance
- Sensitive to initial centroid selection
- Sensitive to outliers
- Works best for spherical-shaped clusters

